In [1]:
from gklearn.utils.graphfiles import loadDataset
import torch


In [77]:
import numpy as np 
Gs, y = loadDataset("/home/bgauzere/work/Recherche/Datasets/MAO/dataset.ds")

atoms = set()
sizes= set()
for G in Gs:
    for n,label in G.nodes(data=True):
        atoms.add(label['atom'])
    sizes.add(G.order())
    
atom_to_onehot = {'C' : [1.,0.,0.], 'N' : [0.,1.,0.],'O' : [1.,0.,0.]}
print(sizes)


{11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27}


In [3]:
t_y = torch.Tensor(y).size()

In [102]:
G =Gs[0]
from utils import encode_onehot

In [103]:
inputs = []
for i,label in G.nodes(data=True):
    #print(i)
    cur_label = atom_to_onehot[label['atom']].copy()
    #cur_label.extend([float(x) for x in label['attributes']])
    #print(label['attributes'])
    #print(cur_label)
    inputs.append(cur_label)
print(len(inputs))
print(G.order())
inputs

11
11


[[0.0, 1.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [0.0, 1.0, 0.0]]

In [104]:
t_data = torch.stack((torch.Tensor(inputs).reshape(-1,len(inputs),3),torch.Tensor(inputs).reshape(-1,len(inputs),3)))

In [105]:
import torch.nn as nn
import torch.nn.functional as F
from layers import GraphConvolution, GraphPooling


class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        #self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout
        self.pooling = GraphPooling()
        self.fc3 = nn.Linear(nhid, nclass)

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        #x = F.relu(self.gc2(x, adj))
        x = self.pooling(x)
        
        x = F.log_softmax(self.fc3(x))
        return x


In [106]:
gcn = GCN(3,10,2,0.5)

In [121]:
def load_MAO():
    import networkx as nx
    from gklearn.utils.graphfiles import loadDataset
    import torch
    
    Gs, y = loadDataset("/home/bgauzere/work/Recherche/Datasets/MAO/dataset.ds")
    t_classes = torch.Tensor(encode_onehot(y))
    max_size = 30
    adjs = []
    inputs = []
    for i,G in enumerate(Gs):
        I = torch.eye(G.order(),G.order())
        A = torch.Tensor(nx.adjacency_matrix(G).todense())
        adj=F.pad(A+I,pad=(0,30-G.order(),0,30-G.order()))
        adjs.append(adj)
        
        f_0 = []
        for _,label in G.nodes(data=True):
            cur_label = atom_to_onehot[label['atom']].copy()
            f_0.append(cur_label)
        
        X = F.pad(torch.Tensor(f_0), pad=(0,0,0,30-G.order()))
        inputs.append(X)
    return inputs,adjs,t_classes

In [125]:
Xs,adjs,y_t = load_MAO()


68

In [110]:
outputs=gcn(X,adj)
outputs

/home/bgauzere/.local/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([-4.4085, -0.0122], grad_fn=<LogSoftmaxBackward>)

tensor([[ 0.5360, -0.3162, -0.1142, -0.2304, -0.2122, -0.4304,  0.8229, -0.5095,
          0.2824, -0.1450],
        [ 0.6286, -0.5747, -0.1262, -0.2673, -0.4405, -0.7413,  1.0537, -0.7807,
          0.2245, -0.2766],
        [ 0.4154, -0.5300,  0.1600, -0.1538, -0.6476, -1.0371,  0.8767, -1.0269,
          0.0515, -0.2068],
        [ 0.5080, -0.7886,  0.1480, -0.1907, -0.8760, -1.3480,  1.1074, -1.2981,
         -0.0064, -0.3384],
        [ 0.4154, -0.5300,  0.1600, -0.1538, -0.6476, -1.0371,  0.8767, -1.0269,
          0.0515, -0.2068],
        [ 0.4154, -0.5300,  0.1600, -0.1538, -0.6476, -1.0371,  0.8767, -1.0269,
          0.0515, -0.2068],
        [ 0.4154, -0.5300,  0.1600, -0.1538, -0.6476, -1.0371,  0.8767, -1.0269,
          0.0515, -0.2068],
        [ 0.4154, -0.5300,  0.1600, -0.1538, -0.6476, -1.0371,  0.8767, -1.0269,
          0.0515, -0.2068],
        [ 0.6286, -0.5747, -0.1262, -0.2673, -0.4405, -0.7413,  1.0537, -0.7807,
          0.2245, -0.2766],
        [ 0.5360, -

None


In [168]:
Y = encode_onehot(y)

t=outputs.reshape(1,-1)

y[0]

0.0

In [170]:
criterion = torch.nn.CrossEntropyLoss()
target = torch.tensor([y[0]]).long()
criterion(t,target)

tensor(4.4085, grad_fn=<NllLossBackward>)

In [181]:
outputs.argmax().item()

1

In [155]:
predicted = torch.tensor([[1.,2.,3.,4.]])
target = torch.tensor([1])
criterion(predicted,target)

tensor(2.4402)

In [151]:
print(predicted.size())
target.size()

torch.Size([1, 4])


torch.Size([1])

In [182]:
A

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [183]:
I

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [ ]:
p=torch.randperm(11)

In [192]:
print(p)
for i in range(11):
    I[i,i] = i

I[p,:][:,p]

tensor([ 0,  3,  2,  8,  5,  1,  7,  9, 10,  6,  4])


tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  5.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.]])

In [198]:
l=[1,2,3,4]
import random
print(random.shuffle(l))
print(l)

None
[1, 4, 3, 2]


In [204]:
l1 = [1,2,3,4]
l2 = ["un","deux","trois","quatre"]
l3 =["one","two","three","four"]

mapIndexPosition = list(zip(l1, l2,l3))
random.shuffle(mapIndexPosition)

In [205]:

for i,j,k in mapIndexPosition:
    print(i,j,k)

3 trois three
4 quatre four
2 deux two
1 un one
